# **Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **GPU**

In [1]:
!nvidia-smi

Tue Aug  8 20:36:42 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# **Install**

In [ ]:
!pip install torchmetrics

# Imports

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, random_split
import torch.backends.cudnn as cudnn

import torchvision
from torchvision.datasets import CIFAR10
from torchvision import transforms as T

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from torchmetrics import Accuracy
from tqdm import tqdm

import glob
import math

# **Init**

In [ ]:
num_cls = 6

# Dataset

## Load dataset

In [ ]:
import os
os.listdir('/content/drive/MyDrive/UCI HAR Dataset/train/Inertial Signals')

['body_acc_x_train.txt',
 'body_acc_y_train.txt',
 'body_acc_z_train.txt',
 'body_gyro_x_train.txt',
 'body_gyro_y_train.txt',
 'body_gyro_z_train.txt',
 'total_acc_x_train.txt',
 'total_acc_y_train.txt',
 'total_acc_z_train.txt']

In [ ]:
def har_dataset(phase):
  x = []
  data_dir = f'/content/drive/MyDrive/UCI HAR Dataset/{phase}/Inertial Signals/*'
  for file_dir in glob.glob(data_dir):
    x.append(np.loadtxt(file_dir))
  y = np.loadtxt(f'/content/drive/MyDrive/UCI HAR Dataset/{phase}/y_{phase}.txt')
  #
  x = torch.FloatTensor(x).permute(1, 2, 0)
  y = torch.LongTensor(y) - 1
  return x, y

In [ ]:
x_train, y_train = har_dataset('train')
x_test, y_test = har_dataset('test')

C:\Users\PC\AppData\Local\Temp\ipykernel_9220\2846403758.py:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:204.)
  x = torch.FloatTensor(x).permute(1, 2, 0)


In [ ]:
x_train.shape, y_train.shape

(torch.Size([7352, 128, 9]), torch.Size([7352]))

In [ ]:
x_test.shape, y_test.shape

(torch.Size([2947, 128, 9]), torch.Size([2947]))

In [ ]:
y_train

tensor([4, 4, 4,  ..., 1, 1, 1])

In [ ]:
torch.unique(y_train)

tensor([0, 1, 2, 3, 4, 5])

## TensorDataset

In [ ]:
train_set = TensorDataset(x_train, y_train)
test_set = TensorDataset(x_test, y_test)

## DataLoader

In [ ]:
train_loader = DataLoader(train_set, batch_size=256, shuffle=True)
test_loader = DataLoader(test_set, batch_size=256, shuffle=False)

In [ ]:
x, y = next(iter(train_loader))
print(x.shape)
print(y.shape)

torch.Size([256, 128, 9])
torch.Size([256])


## Visulaize

In [ ]:
plt.plot(x[5])

# Model

In [ ]:
model = nn.Transformer(9, 3, 3, 3, 36, 0.2, 'gelu', device='cuda')
model

Transformer(
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=9, out_features=9, bias=True)
        )
        (linear1): Linear(in_features=9, out_features=36, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
        (linear2): Linear(in_features=36, out_features=9, bias=True)
        (norm1): LayerNorm((9,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((9,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.2, inplace=False)
        (dropout2): Dropout(p=0.2, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=9, out_features=9, bias=True)
        )
        (linear1): Linear(in_features=9, out_features=36, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
        (li

In [ ]:
model.decoder.layers[2].linear1.bias.shape

torch.Size([36])

In [ ]:
model = nn.Transformer(9, 3, 8, 0).encoder
model

TransformerEncoder(
  (layers): ModuleList(
    (0): TransformerEncoderLayer(
      (self_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=9, out_features=9, bias=True)
      )
      (linear1): Linear(in_features=9, out_features=2048, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (linear2): Linear(in_features=2048, out_features=9, bias=True)
      (norm1): LayerNorm((9,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((9,), eps=1e-05, elementwise_affine=True)
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerEncoderLayer(
      (self_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=9, out_features=9, bias=True)
      )
      (linear1): Linear(in_features=9, out_features=2048, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (linear2): Linear(in_features=2048, out_features=9, bias=True)


In [ ]:
encoder_layer = nn.TransformerEncoderLayer(9, 3, 36, device='cuda')

In [ ]:
model = nn.TransformerEncoder(encoder_layer, 8)
model

TransformerEncoder(
  (layers): ModuleList(
    (0): TransformerEncoderLayer(
      (self_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=9, out_features=9, bias=True)
      )
      (linear1): Linear(in_features=9, out_features=36, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (linear2): Linear(in_features=36, out_features=9, bias=True)
      (norm1): LayerNorm((9,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((9,), eps=1e-05, elementwise_affine=True)
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerEncoderLayer(
      (self_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=9, out_features=9, bias=True)
      )
      (linear1): Linear(in_features=9, out_features=36, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (linear2): Linear(in_features=36, out_features=9, bias=True)
      (n

In [ ]:
model.layers[0].linear1.weight

Parameter containing:
tensor([[ 1.6543e-01, -1.1922e-01, -2.2748e-01,  2.5058e-01,  6.2822e-02,
         -2.4685e-01,  1.5218e-01,  1.6949e-02, -2.1442e-01],
        [ 6.3605e-03, -3.6823e-02, -2.6280e-01, -1.1649e-01,  1.4373e-02,
          1.7927e-01,  1.6837e-01,  2.4524e-01, -5.9314e-02],
        [-2.0802e-01,  2.8351e-01,  3.2805e-01, -2.4618e-01, -2.0542e-01,
          2.1346e-01,  3.2289e-01, -2.7927e-01,  2.6689e-01],
        [-1.0862e-01,  2.1609e-01,  2.7781e-01,  2.2568e-01,  1.0942e-01,
          1.2449e-01, -5.8544e-02,  1.5690e-01, -2.0948e-01],
        [-2.2359e-01,  6.4879e-02, -2.4895e-01,  2.9622e-01, -1.4340e-02,
          2.6352e-01,  9.4844e-02,  9.5034e-02, -6.5691e-02],
        [-2.5453e-01,  2.0097e-01, -3.0952e-01, -2.9513e-01, -1.4800e-01,
          1.1973e-01,  2.5859e-01,  2.9979e-01, -1.7350e-01],
        [ 1.1718e-01, -1.9370e-01,  3.1758e-01,  2.8229e-01, -3.0494e-01,
         -7.6658e-02,  3.0346e-01, -3.6678e-02,  2.9741e-01],
        [ 3.3403e-02, -4.0

In [ ]:
nn.MultiheadAttention()

## HAR Model

In [ ]:
class TransformerModel(nn.Module):

  def __init__(self, d_model, nhead, num_enc, d_feed, dropout, activation):
    super().__init__()
    self.encoder = nn.Transformer(d_model,
                                  nhead,
                                  num_enc,
                                  0,
                                  d_feed,
                                  dropout,
                                  activation,
                                  batch_first=True).encoder
    # cls
    self.fc = nn.LazyLinear(num_cls)
    # input layer
    self.linear0 = nn.LazyLinear(d_model)
    self.bn0 = nn.LazyBatchNorm1d()

  def forward(self, x):
    x = self.bn0(self.linear0(x)).relu()
    y = self.encoder(x)
    # y = y.mean(dim=1)
    y = self.fc(y[:, -1])
    # y = y.mean(dim=1)
    return y

In [ ]:
model = TransformerModel(128, 8, 8, 256, 0.1, 'relu')
# model

C:\Users\PC\anaconda3\envs\howsam-deep\lib\site-packages\torch\nn\modules\lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [ ]:
model(x).shape

torch.Size([256, 6])

## Params

In [ ]:
def num_params(model, k=1e6):
  nums = sum(p.numel() for p in model.parameters())/k
  return nums

In [ ]:
num_params(model, 1e6)

1.062406

# Device

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

# Utils

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


# Functions

In [ ]:
def train_one_epoch(model, train_loader, loss_fn, optimizer, epoch=None):
  model.train()
  loss_train = AverageMeter()
  acc_train = Accuracy().to(device)
  with tqdm(train_loader, unit="batch") as tepoch:
    for inputs, targets in tepoch:
      if epoch is not None:
        tepoch.set_description(f"Epoch {epoch}")
      inputs = inputs.to(device)
      targets = targets.to(device)

      outputs = model(inputs)

      loss = loss_fn(outputs, targets)

      loss.backward()
      nn.utils.clip_grad_norm_(model.parameters(), 0.5)

      optimizer.step()
      optimizer.zero_grad()

      loss_train.update(loss.item())
      acc_train(outputs, targets.int())
      tepoch.set_postfix(loss=loss_train.avg,
                         accuracy=100.*acc_train.compute().item())
  return model, loss_train.avg, acc_train.compute().item()

In [ ]:
def validation(model, test_loader, loss_fn):
  model.eval()
  with torch.no_grad():
    loss_valid = AverageMeter()
    acc_valid = Accuracy().to(device)
    for i, (inputs, targets) in enumerate(test_loader):
      inputs = inputs.to(device)
      targets = targets.to(device)

      outputs = model(inputs)
      loss = loss_fn(outputs, targets)

      loss_valid.update(loss.item())
      acc_valid(outputs, targets.int())
  return loss_valid.avg, acc_valid.compute().item()

# Efficient way for set hyperparams

## Step 1: check forward path

Calculate loss for one batch

In [ ]:
model = TransformerModel(128, 8, 8, 256, 0.1, 'relu').to(device)
loss_fn = nn.CrossEntropyLoss()

x_batch, y_batch = next(iter(train_loader))
outputs = model(x_batch.to(device))
loss = loss_fn(outputs, y_batch.to(device))
print(loss)

## Step 2: check backward path

Select 5 random batches and train the model

In [ ]:
_, mini_train_dataset = random_split(train_set, (len(train_set)-500, 500))
mini_train_loader = DataLoader(mini_train_dataset, 20)

In [ ]:
model = TransformerModel(128, 8, 8, 256, 0.1, 'relu').to(device)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [ ]:
num_epochs = 100
for epoch in range(num_epochs):
  model, _, _ = train_one_epoch(model, mini_train_loader, loss_fn, optimizer, epoch)

## Step 3: select best lr

Train all data for one epoch

In [ ]:
num_epochs = 5
for lr in [0.9, 0.5, 0.2, 0.1, 0.01, 0.001, 0.0001]:
  print(f'LR={lr}')
  model = TransformerModel(128, 8, 256, 0.1, 8, 6).to(device)
  # model = torch.load('model.pt')
  optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=1e-4, momentum=0.9)
  for epoch in range(num_epochs):
    model, _, _ = train_one_epoch(model, train_loader, loss_fn, optimizer, epoch)
  print()

## Step 4: small grid (optional)

Create a small grid based on the WD and the best LR



In [ ]:
num_epochs = 5

for lr in [0.05, 0.04, 0.03, 0.02, 0.01, 0.009, 0.008, 0.007, 0.006, 0.005]:
  for wd in [1e-4, 1e-5, 0.]:
    model = TransformerModel(128, 2, 256, 4, 0.0).to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=wd)
    print(f'LR={lr}, WD={wd}')

    for epoch in range(num_epochs):
      model, loss, _ = train_one_epoch(model, train_loader, loss_fn, optimizer, epoch)
    print()

## Step 5: train more epochs

In [ ]:
model = TransformerModel(64, 8, 64*4, 0.1, 8, 6).to(device)

In [ ]:
lr = 0.1
wd = 1e-4
optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=wd, momentum=0.9)

In [ ]:
loss_train_hist = []
loss_valid_hist = []

acc_train_hist = []
acc_valid_hist = []

best_loss_valid = torch.inf
epoch_counter = 0

In [ ]:
num_epochs = 20

for epoch in range(num_epochs):
  # Train
  model, loss_train, acc_train = train_one_epoch(model,
                                                 train_loader,
                                                 loss_fn,
                                                 optimizer,
                                                 epoch)
  # Validation
  loss_valid, acc_valid = validation(model,
                                     test_loader,
                                     loss_fn)

  loss_train_hist.append(loss_train)
  loss_valid_hist.append(loss_valid)

  acc_train_hist.append(acc_train)
  acc_valid_hist.append(acc_valid)

  if loss_valid < best_loss_valid:
    torch.save(model, f'model.pt')
    best_loss_valid = loss_valid

  print(f'Valid: Loss = {loss_valid:.4}, Acc = {acc_valid:.4}')
  print()

  epoch_counter += 1

# **Plot**

In [ ]:
plt.plot(range(epoch_counter), loss_train_hist, 'r-', label='Train')
plt.plot(range(epoch_counter), loss_valid_hist, 'b-', label='Validation')

plt.xlabel('Epoch')
plt.ylabel('loss')
plt.grid(True)
plt.legend()

In [ ]:
plt.plot(range(epoch_counter), acc_train_hist, 'r-', label='Train')
plt.plot(range(epoch_counter), acc_valid_hist, 'b-', label='Validation')

plt.xlabel('Epoch')
plt.ylabel('Acc')
plt.grid(True)
plt.legend()